# Interaction with the World Homework (#3)
Python Computing for Data Science (c) J Bloom, UC Berkeley, 2016

# 1) Monty: The Python Siri

Let's make a Siri-like program with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

In [7]:
#Wanted to do this one, but PyAudio was being stubborn about not detecting input and output stream. Sigh!
#I will definitely come back to this later, when I am more time at hand. 
#Thus, to atleast practice searching web, I decided to get list of frequencies for the next problem

# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/fall2016/ay250/resources -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.

In [1]:
import numpy
import matplotlib.pyplot as plt
%matplotlib inline

import aifc
import array

In [2]:
### Get list of frequencies from web and store it in array along with their names

from urllib.request import urlopen
response = urlopen("http://www.phy.mtu.edu/~suits/notefreqs.html")
html = response.read()
response.close()

from bs4 import BeautifulSoup
soup = BeautifulSoup(html,"html5lib")


tr = soup.findAll("tr")
nametable = []
freqtable = []
for foo in tr[2:]:
    name = foo.findAll('td')[0].text
    if name.find("#") >0:
        name = name[1:-1]
    nametable.append(name)
    freqtable.append(float(foo.findAll('td')[1].text.strip()))
    
freqtable = numpy.array(freqtable)

In [3]:
def findpeaks(ar, chunk, h = 2):
    '''Find peaks in array (ar) in chuncks of length (chunk) that are higher than the mean in that region by factor\
    of (h)'''
    start = chunk
    size = len(ar)
    peaks = []
    while start < size - start:
        pos = numpy.where(ar[start: start + chunk] == ar[start: start + chunk].max())[0][0]
        if ar[pos + start] > h*ar[start - chunk: start + chunk].mean():
            peaks.append(pos + start)
        start += chunk
    
    return numpy.array(peaks)

def cancel_harmonics(peakpos):
    '''cancel higher harmonics from peaks identified in fourier space'''    
    size = peakpos.size
    present = []
    toskip = []
    for foo in range(size //10):
        if foo in toskip:
            pass
        else:
            ar = peakpos / peakpos[foo]
            harmonics = 1
            #See if next 3 harmonics are identified as minima
            for boo in range(2, 5):
                if abs(peakpos - boo*peakpos[foo]).min() < 0.05*peakpos[foo]:
                    pass
                else: harmonics = 0 
            #If harmonics are identified, set all higher harmonics to 0
            if harmonics == 1:
                present.append(foo)
                for boo in range(2, 20):
                    toskip.append(numpy.where(abs(ar - boo) == abs(ar - boo).min())[0][0])

    return numpy.array(present)

In [4]:

for files in ["./hw3_sound_files/sound_files/A4_PopOrgan.aif",
             "./hw3_sound_files/sound_files/C4+A4_PopOrgan.aif",
             "./hw3_sound_files/sound_files/F3_PopOrgan.aif",
             "./hw3_sound_files/sound_files/F4_CathedralOrgan.aif"]:
             
    #Read in the files, convert to array and fourier transform them
    wf = aifc.open(files)
    frames = wf.readframes(wf.getnframes())
    framear = numpy.fromstring(frames, dtype=numpy.uint32).byteswap()#Played with different dtypes
    framef = abs(numpy.fft.fft(framear))

    #Find corresponding frequencies
    freq = numpy.fft.fftfreq(framef.size)*wf.getframerate()

    #Find peaks in FFT. Played with height and chunk
    peakpos = findpeaks(framef, chunk = 2000, h = 3)
    present = cancel_harmonics(freq[peakpos])
    freqfound = freq[peakpos[present]]

    #See if the frequency found is correct, i.e. one of the known frequencies
    correctfreq = []
    namecorrectfreq = []
    tolerance = 1
    while len(correctfreq) <= 0:
        tolerance += 1
        for foo in freqfound:
            if abs(freqtable - foo).min() < tolerance:
                correctfreq.append(foo)
                namecorrectfreq.append(nametable[numpy.where(abs(freqtable - foo) == abs(freqtable - foo).min())[0][0]])
    print(correctfreq, namecorrectfreq)


[439.25000000000006] ['A4']
[261.5625, 439.25000000000006] ['C4', 'A4']
[174.56250000000003] ['F3']
[350.3125] ['F4']


In [5]:
for i in range(1,13):

    #Read in the files, convert to array and fourier transform them
    wf = aifc.open("./hw3_sound_files/sound_files/%d.aif"%i)

    frames = wf.readframes(wf.getnframes())
    framear = numpy.fromstring(frames, dtype=numpy.uint32).byteswap()#Played with different dtypes
    framef = abs(numpy.fft.fft(framear))

    #Find corresponding frequencies
    freq = numpy.fft.fftfreq(framef.size)*wf.getframerate()

    #Find peaks in FFT. Played with height and chunk
    peakpos = findpeaks(framef, chunk = 2000, h = 3)
    present = cancel_harmonics(freq[peakpos])
    freqfound = freq[peakpos[present]]

    #See if the frequency found is correct, i.e. one of the known frequencies
    correctfreq = []
    namecorrectfreq = []
    tolerance = 1
    while len(correctfreq) <= 0:
        tolerance += 1
        for foo in freqfound:
            if abs(freqtable - foo).min() < tolerance:
                correctfreq.append(foo)
                namecorrectfreq.append(nametable[numpy.where(abs(freqtable - foo) == abs(freqtable - foo).min())[0][0]])
    print(correctfreq, namecorrectfreq)


[261.5625, 391.25000000000006] ['C4', 'G4']
[174.56250000000003] ['F3']
[884.00000000000011] ['A5']
[261.5625] ['C4']
[879.93749999999989] ['A5']
[523.0625] ['C5']
[587.75] ['D5']
[348.5] ['F4']
[195.9375] ['G3']
[130.75] ['C3']
[1319.8125, 1398.1875000000002] ['E6', 'F6']
[1765.625] ['A6']
